# Fetch BTC 2020 to 2024 from test net api

In [1]:
from dataclasses import dataclass
from sqlite3 import DataError
from binance.client import Client
import pandas as pd
import datetime

In [2]:
# Replace with your own Binance API key and secret
api_key = 'dd1a2703da01c24ccdbe10b2cbc9600a6218c41434d4453916766e1e817aabfe'
api_secret = '2acb5c37db3c5646fdadb366cacb5d9a11ffe05d7b4d01dfb1b3047f2a2ea09a'

# Initialize the Binance client with increased timeout
client = Client(api_key, api_secret, {"timeout": 60})

In [9]:
# Function to get OHLC data in chunks
def get_ohlc_data(symbol, interval, start_str, end_str):
    """
    Get OHLC data from Binance in chunks.

    :param symbol: String, the symbol of the trading pair (e.g., 'BTCUSDT')
    :param interval: String, the interval for candlesticks (e.g., '1m', '1h', '1d')
    :param start_str: String, the start date in 'YYYY-MM-DD' format
    :param end_str: String, the end date in 'YYYY-MM-DD' format
    :return: DataFrame, OHLC data
    """
    # Fetch historical klines using the Binance API
    klines = client.get_historical_klines(symbol, interval, start_str, end_str)
    
    df = pd.DataFrame(klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume', 
        'close_time', 'quote_asset_volume', 'number_of_trades', 
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
     # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    # Set timestamp as index
    df.set_index('timestamp', inplace=True)
    
    # Drop unnecessary columns
    df.drop(columns=['close_time', 'quote_asset_volume', 'number_of_trades', 
                     'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'], inplace=True)
    
    return df

In [10]:
# Example usage
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1MINUTE
start_date = "2020-01-01"  # Example: fetch 1 minute klines for the last day up until now
end_date = "2024-07-04" # End date as current date and time
df = get_ohlc_data(symbol, interval, start_date, end_date)
print(df)


                               open            high             low  \
timestamp                                                             
2020-01-01 00:00:00   7195.24000000   7196.25000000   7183.14000000   
2020-01-01 00:01:00   7187.67000000   7188.06000000   7182.20000000   
2020-01-01 00:02:00   7184.41000000   7184.71000000   7180.26000000   
2020-01-01 00:03:00   7183.83000000   7188.94000000   7182.49000000   
2020-01-01 00:04:00   7185.54000000   7185.54000000   7178.64000000   
...                             ...             ...             ...   
2024-07-03 23:56:00  60292.13000000  60292.14000000  60292.13000000   
2024-07-03 23:57:00  60292.14000000  60292.14000000  60252.00000000   
2024-07-03 23:58:00  60252.01000000  60252.01000000  60230.01000000   
2024-07-03 23:59:00  60230.01000000  60230.02000000  60208.57000000   
2024-07-04 00:00:00  60208.57000000  60250.00000000  60208.57000000   

                              close       volume  
timestamp                

OSError: Cannot save file into a non-existent directory: 'C:\Users\MTariq\Desktop\New folder'

In [11]:

# Save DataFrame to CSV file in the specified path
output_path = r'C:\Users\MTariq\Desktop\TestNet Project\ohlc_1m_data_last_day.csv'
df.to_csv(output_path)